In [ ]:
import os
os.chdir(os.path.join(os.path.abspath(os.path.expanduser("~")), "Documents", "Personal", "Test"))
from scripts.tools.issuetracker import IssuetrackerAPI as _ISAPI, Issue
import pickle

import datetime, dateutil
from_date = datetime.datetime(2016, 6, 1, tzinfo=dateutil.tz.tzutc())
today =datetime.date.today()
to_date = datetime.datetime(today.year, today.month, today.day, tzinfo=dateutil.tz.tzutc())


class IssuetrackerAPI(_ISAPI):
    
    def make_pickle_cache(self, *args):
        issues = super().download_issues(*args)
        with open("gantt_dummy.pkl", 'wb') as f:
            pickle.dump(issues, f)
        return issues
    def download_issues(self, *args):
        if not os.path.exists("gantt_dummy.pkl"):
            return self.make_pickle_cache(*args)
        else:
            with open("gantt_dummy.pkl", 'rb') as f:
                try:
                    issues = pickle.load(f)
                except (pickle.PickleError, OSError, EOFError):
                    pass
                else:
                    return issues
            return self.make_pickle_cache(*args)
        return issues


In [ ]:
from officelib import wordlib
from scripts.tools.issuetracker.frs_update import main
import shutil
word = wordlib.Word()
pdfpth = "pdfs6"
try:
    word.ActiveDocument.Close(False)
except:
    pass
try:
    shutil.rmtree(pdfpth)
except FileNotFoundError:
    pass
print("Begin Main", flush=True)
res = main("issue.pbsbiotech.com", 'nstarkweather', 'kookychemist', 'pbssoftware', "2016-6-1", 
     "today", 'Hello v3.0 FRS Update', pdfpth, "FRS Update", 
     "IS00038B2 Rev A Functional Requirements Specification Updates")
#os.startfile('"%s"'%res)
print()
print("Done")

# 3/2/2017 and 4/5/2017

In [ ]:
from officelib import wordlib
from scripts.tools.issuetracker.frs_update import main
import shutil
word = wordlib.Word()
pdfpth = "Software_FRS_170405"
try:
    word.ActiveDocument.Close(False)
except:
    pass
try:
    shutil.rmtree(pdfpth)
except FileNotFoundError:
    pass
print("Begin Main", flush=True)
res = main("issue.pbsbiotech.com", 'nstarkweather', 'kookychemist', 'pbssoftware', None, 
     "today", 'Hello v3.0 FRS Update', pdfpth, "FRS Update", 
     "IS00038B2 Rev A Functional Requirements Specification")
#os.startfile('"%s"'%res)
print()
print("Done")

# 5/4/2017

In [1]:
from scripts.tools.issuetracker.frs_update import main
import shutil
import datetime

def bt():
    return datetime.date.today().strftime("%y%m%d")
    
pdfpth = "Software_FRS_" + bt()

try:
    shutil.rmtree(pdfpth)
except FileNotFoundError:
    pass
print("Begin Main", flush=True)
res = main("issue.pbsbiotech.com", 'nstarkweather', 'kookychemist', 'pbssoftware', None, 
     "today", 'Hello v3.0 FRS Update', pdfpth, "FRS Update", 
     "IS00038C2 Rev A Functional Requirements Specification")
#os.startfile('"%s"'%res)
print()
print("Done")

Begin Main

Logging into Issuetracker...Downloading projects...

Got update: PGS_BEGIN_PROCESS_PDFS 
Creating PDF Index...
Creating Word Document...
Creating index 84 of 84                    
Got update: PGS_IPG_FLAG_WAITING 
Got update: PGS_IPG_FLAG_WAITING 2
Got update: PGS_IPG_FLAG_WAITING 3
Got update: PGS_IPG_FLAG_WAITING 4
Got update: PGS_IPG_FLAG_WAITING 5
Got update: PGS_IPG_FLAG_WAITING 6
Got update: PGS_IPG_FLAG_WAITING 7
Got update: PGS_IPG_FLAG_WAITING 8
Got update: PGS_IPG_FLAG_WAITING 9
Got update: PGS_IPG_FLAG_WAITING 10
Got update: PGS_IPG_FLAG_WAITING 11
Got update: PGS_IPG_FLAG_WAITING 12
Got update: PGS_IPG_FLAG_WAITING 13
Got update: PGS_IPG_FLAG_WAITING 14
Got bad gateway error for issue #2881: retrying
Got update: MPD_ALL_COMPLETE 
Got update: PGS_PDFTK_INFO_BEGIN 
Getting PDF Page Info (81/81)                    
Got update: PGS_PDFTK_INFO_DONE 
Got update: PGS_PROCESS_PDFS_DONE 
Got update: PGS_PREPARE_POSTSCRIPT 

KeyError: Issue(author=User(name='James Small', id=42), custom_fields={...}, sprint_milestone=ResourceWithID(name='3.0', id=52), category=None, status=ResourceWithID(name='Unit Test', id=10), company=None, created_on=datetime.datetime(2015, 10, 29, 21, 10, 11, tzinfo=tzutc()), description='*+FRS2865+*\r\n!Alarm_Editor.png!\r\n*Image 1*: Alarm Editor Panel\r\n\r\n!Alarm_Editor_Numbered.png!\r\n*Image 2*: Numbered Mockup\r\n\r\n* *FRS2865.1:* Appearance\r\n** Item 1 is a path indicator labeled Filename\r\n** Item 2 is a Boolean button labeled Open\r\n** Item 3 is a Boolean button labeled Save\r\n** Item 4 is a Boolean button labeled Make Active\r\n** Item 5 is a Boolean button labeled Reset\r\n** Item 6 is an array control containing alarm condition enum followed by a cluster of checkboxes labeled at the top of each column with N, A, and E, with a key at the bottom defining each, N: Notify, A: Audible, E: Email.\r\n** Item 7 is an array control of strings. The control is labeled Email Li